<a href="https://colab.research.google.com/github/P2Enjoy/ia-web3.fr/blob/main/SpeedWatch_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install all requirements (run once)

# System
import locale, os
locale.getpreferredencoding = lambda: "UTF-8"
!pip install --upgrade torch git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/accelerate.git bitsandbytes==0.41.0 tensorflow==2.13.0 onnxruntime-gpu==1.15.1
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Application for ponctuation
!pip install git+https://github.com/P2Enjoy/deepmultilingualpunctuation.git
!pip install scikit-learn
!pip install nltk

!pip install pytorch-crf==0.7.2 youtube_transcript_api==0.6.1 langchain sentence_transformers

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-rldkypym
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-rldkypym
  Resolved https://github.com/huggingface/transformers.git to commit 66c240f3c950612fa05b2e14c85d4b86c88e473e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-7npi__t2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-7npi__t2
  Resolved https://github.com/huggingface/accelerate.git to commit 49cb83a423f2946059117d8bb39b7c8747d29d80
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/P2Enjoy/deepmultilingualpunctuation.git to /tmp

# Youtube processing

In [2]:
#@title Put the link of the video to summarize:
video_link = "https://www.youtube.com/watch?v=ku6bCLx9roY" #@param {type:"string"}

In [3]:
#@title Extract the video transcript for all youtube video
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

def YTVideoToText(video_link):
    # Extract video ID from the video_link
    parsed_url = urlparse(video_link)
    video_id = None
    if parsed_url.netloc == 'www.youtube.com':
        video_id = parse_qs(parsed_url.query).get('v', [None])[0]
    elif parsed_url.netloc == 'youtu.be':
        video_id = parsed_url.path.split('/')[1]

    if not video_id:
        raise ValueError("Invalid YouTube video link or missing video ID")

    # Fetching video transcript
    try:
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
        transcript = YouTubeTranscriptApi.get_transcript(video_id= video_id, languages= ['en', 'fr'])
    except Exception as e:
        raise ValueError("Error fetching transcript: " + str(e))

    # Iterating throughout and adding all text together (recovering punctuations)
    flat_transcript = ""
    for tu in transcript:
        flat_transcript += ' ' + tu['text']

    return flat_transcript

try:
    video_full_text = YTVideoToText(video_link)
    print(video_full_text)
except ValueError as ve:
    print(ve)

video_full_text = YTVideoToText(video_link)

 en 1927 le diplomate roumain mathilagica publie esthétique des proportions dans la nature et dans les arts dans lequel il introduit le nombre d'or comme marqueur universel de la beauté il s'appuie sur les travaux de philosophalement du XIXe siècle comme Gustave Fechner et son rectangle d'or ou adolesing et sa section d'or eux-mêmes se sont appuyé sur les recherches du mathématicien de la Renaissance Lucas passioli et de sa divine proportion qui lui s'est fortement inspiré des travaux d'Euclide et de son partage en moyenne et extrême raison bref l'histoire du nombre d'or est tumultueuse et ça tombe bien j'ai deux minutes pour en parler j'ai ouvert cette chaîne youtube il y a 8 ans et il y a ce sujet que j'ai à peine évoqué et qui semble chez certains au cœur des mathématiques le nombre d'or pour ceux qui ne connaissent pas le nombre d'or c'est un nombre qui est partout et qui est ni plus ni moins que la clé pour appréhender la beauté du monde qui nous entoure pour comprendre de quoi il

In [4]:
#@title Restore the ponctuation

from deepmultilingualpunctuation import PunctuationModel

punctuationModel = PunctuationModel(model= "kredor/punctuate-all")
video_restored_text = punctuationModel.restore_punctuation(video_full_text)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="none"` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


# Summarization

In [5]:
#@title Load the model into memory
# Load model directly
from transformers import AutoTokenizer, pipeline

sum_model_id = "csebuetnlp/mT5_multilingual_XLSum" #@param {type:"string"}
sum_tokenizer = AutoTokenizer.from_pretrained(sum_model_id)
summarizeryt = pipeline('summarization', model=sum_model_id, device_map="auto")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [6]:
#@title Join sentences in paragraphs of less the specified token size
summarize_token_width = 512 #@param {type:"slider", min:256, max:8192, step:256}

from langchain.text_splitter import CharacterTextSplitter
splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    sum_tokenizer, separator=f'\. ', chunk_size=summarize_token_width, chunk_overlap=0
)

In [7]:
#@title Extract the summary
def do_sum(sentences):
  # summarize text
  summarized_text = ''
  for sentence in sentences:
      out = summarizeryt(sentence, min_length=30, do_sample=False)
      out = out[0]
      out = out['summary_text']
      summarized_text += f'.\n{out}'

  # returning summary
  return summarized_text;

video_summarized_text = do_sum(splitter.split_text(video_restored_text))

In [8]:
#@title Show the summarized text of the video
video_summarized_text

".\nDans notre série de lettres de journalistes africains, l'écrivaine et romancière Adaobi Tricia Nwaubani se penche sur le nombre d'or comme marqueur universel de la beauté du monde qui nous entoure..\nDans notre série de lettres de journalistes africains, l'écrivaine et romancière Adaobi Tricia Nwaubani réfléchit à la différence entre le nombre d'or et la beauté..\nDans notre série de lettres de journalistes africains, l'écrivaine et romancière Adaobi Tricia Nwaubani se penche sur le degré de perfection de la beauté..\nSi je cherche des rectangles d'un format précis dans une peinture, pour peu que ce que j'étudie est un minimum de complexité..\nDans notre série de lettres de journalistes africains, l'écrivaine et romancière Adaobi Tricia Nwaubani s'est entretenu avec des artistes tels que Leonardo da Vinci, Gustav Klimt et Picasso..\nPourquoi le nombre d'or a été considéré comme un nombre à part entière? Nous l'avons tous entendu..\nDans notre série de lettres de journalistes africa

# Harmonizing the text output


In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

llm_tokenizer = AutoTokenizer.from_pretrained("OpenAssistant/llama2-13b-orca-8k-3319", use_fast=False)
llm_model = AutoModelForCausalLM.from_pretrained("OpenAssistant/llama2-13b-orca-8k-3319", device_map="auto", load_in_4bit=True, offload_folder='./.cache')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [18]:
system_message = "You are an insightful, adaptive, and respectful assistant, equipped with the ability to seamlessly dive deep into a user-preferred subject. You should provide detailed and relevant information on the chosen topic, enhancing the user's understanding and interest. Always answer as helpfully as possible while ensuring your responses are positive and respectful. If a user's query is not clear or not factually coherent within the context of the preferred subject, please explain why instead of providing inaccurate information. Strive to be unbiased on other topics but accept to bias your responses towards the user-selected subject. In case you don't know the answer to a question within the chosen field, please admit it rather than sharing false information."
user_prompt = f"""Summarize the following text with an accent on history: ```{video_summarized_text}```"""
prompt = f"""<|system|>{system_message}</s><|prompter|>{user_prompt}</s><|assistant|>"""

In [19]:
inputs = llm_tokenizer(prompt, return_tensors="pt").to("cuda")
output = llm_model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=2048)
print(llm_tokenizer.decode(output[0], skip_special_tokens=True))

You are an insightful, adaptive, and respectful assistant, equipped with the ability to seamlessly dive deep into a user-preferred subject. You should provide detailed and relevant information on the chosen topic, enhancing the user's understanding and interest. Always answer as helpfully as possible while ensuring your responses are positive and respectful. If a user's query is not clear or not factually coherent within the context of the preferred subject, please explain why instead of providing inaccurate information. Strive to be unbiased on other topics but accept to bias your responses towards the user-selected subject. In case you don't know the answer to a question within the chosen field, please admit it rather than sharing false information.Summarize the following text with an accent on history: ```.
Dans notre série de lettres de journalistes africains, l'écrivaine et romancière Adaobi Tricia Nwaubani se penche sur le nombre d'or comme marqueur universel de la beauté du mond